In [1]:
# youtube8m のディレクトリに移動
import os
while os.getcwd().split('/')[-1] != 'youtube8m': os.chdir('..')
os.getcwd()

'/home/jupyter/ASLOpenProject/youtube8m'

In [2]:
import tensorflow as tf
from video.data import read_dataset

In [3]:
tf.enable_eager_execution()

# カテゴリ一覧

In [4]:
!gsutil cp gs://asl-mixi-project-bucket/data/youtube-8m/vocabulary.csv .

Copying gs://asl-mixi-project-bucket/data/youtube-8m/vocabulary.csv...
/ [1 files][  3.4 MiB/  3.4 MiB]                                                
Operation completed over 1 objects/3.4 MiB.                                      


In [5]:
import csv

label_map = {}
with open('./vocabulary.csv', 'r') as c:
    reader = csv.reader(c)
    next(reader, None)
    for r in reader:
        label_map[r[0]] = r[3]

# 評価データセットの読み込み

In [6]:
data = read_dataset(
    'gs://asl-mixi-project-bucket/data/youtube-8m/valid/*.tfrecord',
    tf.estimator.ModeKeys.EVAL,
    batch_size=5
).shuffle(1024)
data_iter = iter(data)

# 推論

In [7]:
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials
import json

PROJECT = 'qwiklabs-gcp-ml-83914069970b'
MODEL_NAME = 'youtube8mkojo'
VERSION = 'v1'

credentials = GoogleCredentials.get_application_default()
api = discovery.build(
    "ml",
    "v1",
    credentials = credentials,
    discoveryServiceUrl = "https://storage.googleapis.com/cloud-ml/discovery/ml_v1_discovery.json"
)

def convert_data(data_iter):
    batch, labels = next(data_iter)
    batch.pop('id')
    feature = [{'mean_rgb': r.numpy().tolist(), 'mean_audio': a.numpy().tolist()} for r, a in zip(batch['mean_rgb'], batch['mean_audio'])]
    labels = [[label_map[str(i)] for i, v in enumerate(label) if v] for label in labels]
    return feature, labels

def show_predict(prediction, label):
    print('labels:', label)
    for i, index in enumerate(prediction["predicted_topk"]):
        print("{}th({}:{}) probality is {}".format(
            (i+1),
            index,
            label_map["{}".format(index)],
            prediction['probabilities'][index]
        ))
    print('')

def predict(data_iter, model_name=MODEL_NAME, version=VERSION):
    feature, labels = convert_data(data_iter)
    request_data = {"instances": feature}
    parent = "projects/{}/models/{}/versions/{}".format(PROJECT,MODEL_NAME, VERSION)
    response = api.projects().predict(body = request_data, name = parent).execute()
    for p, l in zip(response['predictions'], labels):
        show_predict(p, l)
    return None

In [8]:
predict(data_iter, 'youtube8mkojo', 'v1')

labels: ['Square dance']
1th(8:Dance) probality is 0.9692925214767456
2th(835:Aerobics) probality is 0.0675254762172699
3th(206:Ballet) probality is 0.008715659379959106
4th(6:Performance art) probality is 0.007174193859100342
5th(812:Dance studio) probality is 0.0058457255363464355

labels: ['Art', 'Drawing', 'Mehndi']
1th(49:Art) probality is 0.9657893180847168
2th(80:Drawing) probality is 0.9537800550460815
3th(265:Sketch (drawing)) probality is 0.8648155927658081
4th(5:Cartoon) probality is 0.46875593066215515
5th(694:Pencil) probality is 0.3122357428073883

labels: ['Game', 'Video game', 'Strategy video game', 'League of Legends']
1th(111:League of Legends) probality is 0.9994308948516846
2th(0:Game) probality is 0.9622862339019775
3th(42:Strategy video game) probality is 0.9238297939300537
4th(1:Video game) probality is 0.6591159105300903
5th(3143:Riven) probality is 0.05941206216812134

labels: ['Guitar', 'Accordion']
1th(9:Guitar) probality is 0.928152322769165
2th(4:Musician) 